In [1]:
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time

In [2]:
model_path = r"D:\vs code\python\DeepLearning\Projects\Facial_landmark_detection\models\face_landmarker.task"


mp_face_mesh = solutions.face_mesh

In [3]:
def draw_landmarks_on_image(image,detection_results):
    face_landmarks_list = detection_results.face_landmarks
    annotated_image = np.copy(image)

    try:
        # looping through the detected face to visualise
        for index in range(len(face_landmarks_list)):
            face_landmarks = face_landmarks_list[index]

            # Draw landmarks
            face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            face_landmarks_proto.landmark.extend([
                landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
            ])

        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_tesselation_style())
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_contours_style())
        solutions.drawing_utils.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks_proto,
            connections=mp.solutions.face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp.solutions.drawing_styles
            .get_default_face_mesh_iris_connections_style())
    except Exception as e:
        print(e)

    return annotated_image


In [4]:
base_options = python.BaseOptions(model_asset_path=model_path)

options = vision.FaceLandmarkerOptions(base_options=base_options,output_face_blendshapes=True,output_facial_transformation_matrixes=True,num_faces=1,running_mode=vision.RunningMode.VIDEO)

In [5]:

camera = cv2.VideoCapture(0)
with vision.FaceLandmarker.create_from_options(options) as landmarker:
    while True:
        # Read data from the webcam
        success, image = camera.read()
        if not success:
            print("Ignoring empty camera frame...")
            continue
        image.flags.writeable = False
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        face_landmarker_result = landmarker.detect_for_video(mp_image,int(time.time() * 1000))
        annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), face_landmarker_result)
        if not annotated_image.any():
            print("no face detected...")
        else:
            cv2.imshow("window",cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
            if cv2.waitKey(1) & 0xFF == ord("q"):  
                break

cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated with a value
cannot access local variable 'face_landmarks_proto' where it is not associated w

In [ ]:
cv2.destroyAllWindows()